In [ ]:
import numpy as np
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo
import pandas as pd
import matplotlib.animation as animation
from matplotlib.colors import LightSource
from matplotlib import cm
import matplotlib.colors
from matplotlib.gridspec import GridSpec

# Import colormaps and limits
import sys
sys.path.append('./')
from colormaps import *

In [2]:
data_dir = '../../Zenodo/model_output'

# LA_M, LA_H
M_LAN = xr.open_dataset(f'{data_dir}/LA_M.nc')
H_LAN = xr.open_dataset(f'{data_dir}/LA_H.nc')

# QU_M, PI_M, PL_M
M_QUN = xr.open_dataset(f'{data_dir}/QU_M.nc')
M_PIN = xr.open_dataset(f'{data_dir}/PI_M.nc')
M_PLN = xr.open_dataset(f'{data_dir}/PL_M.nc')

# QU_H, PI_H, PL_H
H_QUN = xr.open_dataset(f'{data_dir}/QU_H.nc')
H_PIN = xr.open_dataset(f'{data_dir}/PI_H.nc')
H_PLN = xr.open_dataset(f'{data_dir}/PL_H.nc')

# QU_H, PI_H, PL_H
H_QUN_retuned = xr.open_dataset(f'{data_dir}/retuned/QU_H_retuned.nc')
H_PIN_retuned = xr.open_dataset(f'{data_dir}/retuned/PI_H_retuned.nc')
H_PLN_retuned = xr.open_dataset(f'{data_dir}/retuned/PL_H_retuned.nc')


spinup = xr.open_dataset(f'{data_dir}/spinup.nc')

In [ ]:
files = [M_LAN, M_QUN, M_PIN, M_PLN, H_LAN, H_QUN, H_PIN, H_PLN]
labels = ['LADDIE', 'QUADRATIC', 'PICO', 'PLUME']
colors = ['C3', 'C9', 'C8', 'C7']
printxy = []

fig = plt.figure(figsize=(13,7), dpi=200)

gs = GridSpec(4, 7, height_ratios=[1, 1,.1,.2],width_ratios=[1, .1,1,1,1,.1,1],hspace=0.2, wspace=0.2)

# t = 0
ax0 = fig.add_subplot(gs[0])                            
ax1 = fig.add_subplot(gs[2], sharex=ax0, sharey=ax0)
ax2 = fig.add_subplot(gs[3], sharex=ax0, sharey=ax0)
ax3 = fig.add_subplot(gs[4], sharex=ax0, sharey=ax0)
ax0m = fig.add_subplot(gs[6], sharey=ax0)

# t = 10
ax4 = fig.add_subplot(gs[7])                            
ax5 = fig.add_subplot(gs[9], sharex=ax4, sharey=ax4)
ax6 = fig.add_subplot(gs[10], sharex=ax4, sharey=ax4)
ax7 = fig.add_subplot(gs[11], sharex=ax4, sharey=ax4)
ax1m = fig.add_subplot(gs[13], sharey=ax0)

caxm = fig.add_subplot(gs[23:25])

axs = [ax0,ax1,ax2,ax3,ax4,ax5,ax6,ax7]

for axes in [ax0,ax1,ax2,ax3,ax0m]:
    plt.setp(axes.get_xticklabels(), visible=False)

for axes in [ax1,ax2,ax3,ax5,ax6,ax7,ax0m, ax1m]:
    plt.setp(axes.get_yticklabels(), visible=False)

for i, ds in enumerate(files):

    # Select time slice and x slice
    ds = ds.sel(time=0, x = slice(30000,250000))
        
    newx = ds.y[::-1]/1000
    newy = ds.x/1000
    newBMB = ds.BMB.T*-1
    newmask = ds.mask.T
        
    # Compute deep melt average [m/yr] and integrated melt flux [Gt/yr]
    deepmelt = np.nanmean((xr.where(np.logical_and(ds.mask==4, ds.Hs - ds.Hi < -300), 1, np.nan) * - ds.BMB).T)
    intmelt = np.nansum(newBMB*2000*2000*918*1e-12) # ==> melt in m ice per year * area per gridcell * rhoice * 1e-12 [conversion kg to Gt]

    # Plot melt 
    immelt = axs[i].pcolormesh(newx, newy,    newBMB,                                                     cmap=cmap_BMB,      norm=norm_BMB)
    axs[i].text(-38, 30000/1000+5, f'{deepmelt:.2f} m/yr, {intmelt:.2f} Gt/yr', fontsize=8)

    # Plot tuning contour
    ds_contour = ds.sel(x=slice(70000,250000))
    axs[i].contour(ds_contour.y[::-1]/1000, ds_contour.x/1000, xr.where(ds_contour['mask'].T>3, ds_contour.Hs.T - ds_contour.Hi.T, 10), [-300], linewidths=2,vmin=-301, vmax=-299, colors='k', zorder=1)
        
    # Plot ocean and grounded mask
    axs[i].pcolormesh(newx, newy,    xr.where(newmask==1, 1, np.nan),                            cmap=cmap_ocean,    norm=norm_ocean)
    axs[i].pcolormesh(newx, newy,    xr.where(np.logical_or(newmask==3,newmask==7), 1, np.nan),  cmap=cmap_ground,   norm=norm_ground)

    if i < 4:
        # print(np.max(xr.where(newmask==4, newBMB, np.nan)))
        ax0m.plot(xr.where(newmask==4, newBMB, np.nan).sel(y=0),newy, lw=2, label=labels[i], color=colors[i], alpha=0.7)
        #ax0m.plot(xr.where(newmask==4, newBMB, np.nan).sel(y=-20000),newy, lw=2, label=labels[i], color=colors[i], alpha=0.7, ls='dashed')
        # ax0m.plot(xr.where(newmask==4, newBMB, np.nan).sel(y=20000),newy, lw=2, label=labels[i], color=colors[i], alpha=0.9, ls='dotted')

    else: 
        # print(np.max(xr.where(newmask==4, newBMB, np.nan)))
        ax1m.plot(xr.where(newmask==4, newBMB, np.nan).sel(y=0),newy, lw=2, color=colors[i-4], alpha=0.7)
        # ax1m.plot(xr.where(newmask==4, newBMB, np.nan).sel(y=-30000),newy, lw=2, color=colors[i-4], alpha=0.7)
        # ax1m.plot(xr.where(newmask==4, newBMB, np.nan).sel(y=20000),newy, lw=2, color=colors[i-4], alpha=0.9, ls='dotted')

    ax0m.set_xlim(-10,120)
    ax1m.set_xlim(-10,120)
    ax0m.grid(True, alpha=0.3, ls='dashed')
    ax1m.grid(True, alpha=0.3, ls='dashed')
    ax1m.set_xlabel('Sub-shelf melt [m/yr]')
    ax0m.set_aspect(0.8)
    ax1m.set_aspect(0.8)
    ax0m.text(25,220,'LADDIE',  weight='bold',c=colors[0])
    ax0m.text(25,200,'QUAD',    weight='bold',c=colors[1])
    ax0m.text(25,180,'PICO',    weight='bold',c=colors[2])
    ax0m.text(25,160,'PLUME',   weight='bold',c=colors[3])

    #axs[0].vlines(0, 50, 240, ls='dashed', color='white')
    if i < 4:
        axs[i].set_title(f'{labels[i]}', color=colors[i], weight='bold')
        axs[i].set_aspect(0.5)
        axs[i].set_xticks([-40,0,40])
        axs[i].tick_params(axis='both', labelsize=10)

        axs[i].vlines(0, 50, 240, ls='dashed', color='white',alpha=0.4)
        # axs[i].vlines(-20, 50, 240, ls='dashed', color='white',alpha=0.4)

    else:
        axs[i].set_xlabel('[km]', fontsize=10)
        axs[i].set_aspect(0.5)
        axs[i].set_xticks([-40,0,40])

        axs[i].tick_params(axis='both', labelsize=10)

        axs[i].vlines(0, 50, 240, ls='dashed', color='white',alpha=0.4)

axs[0].text(-34  , 90,'GL'   , size=12  ,weight='bold',ha='center', va='center')
axs[0].text(-29  , 80,'w'    , size=9   ,weight='bold',ha='center', va='center')
axs[0].text( 33  , 90,'GL'   , size=12  ,weight='bold',ha='center', va='center')
axs[0].text( 38  , 80,'e'    , size=9   ,weight='bold',ha='center', va='center')

axs[0].text(-80, 140, f'moderate-melt', fontsize=12, rotation=90,ha='center', va='center')
axs[4].text(-80, 140, f'high-melt', fontsize=12, rotation=90,ha='center', va='center',)


# Take colors at regular intervals spanning the colormap.
colors = cmap_ground(np.linspace(0, 1, 17))
cplot=colors[0]
# Pannel labels
labmod = ['(a)', '(b)', '(c)', '(d)', '(e)']
labhig = [ '(f)', '(g)', '(h)', '(i)', '(j)']

for j in range(4):
    axs[j].text(-38,55,labmod[j],weight='bold')
    axs[j+4].text(-38,55,labhig[j],weight='bold')
ax0m.text(-2,55,labmod[4],weight='bold')
ax1m.text(-2,55,labhig[4],weight='bold')

for ax in [ax0, ax4]:
    ax.set_ylabel('[km]')
    ax.set_yticks([50,100,150,200, 250])

cbar = plt.colorbar(immelt, ax = caxm, ticks=[-10,-1, 0, 1, 10, 100], label='sub-shelf melt [m/yr]', extend='both', orientation='horizontal', pad=.1, fraction=1)
cbar.ax.set_xticklabels([-10,-1, 0, 1, 10, 100], fontsize=10);
cbar.set_label('Sub-shelf melt [m/yr]', fontsize=10)
caxm.set_axis_off()

fig.savefig('fig05.pdf',dpi=300,bbox_inches = 'tight')